# IA par renforcement from scratch

Le but de cette ia est de trouver par la méthode du renforcement.

## Règle du jeu

Le jeu consiste à remplir le plus de case possible sur le plateau avec O voici les contraintes :
Quand O est posé on ne peut plus poser de O sur la même ligne / colonne et diagonales.

## Solution

### Choix de librairie
- copy
Pour réaliser de la recherche en profondeur sur mon arbre de mes solutions générés.

- concurrent.futures
Pour faire du multithreader afin d'accélérer le temps de calcul du programme.

### Explication des méthodes de la classe plateau

- __init__(self, n): 

Le constructeur de la classe Plateau. Il initialise un plateau de taille n avec des cases vides.

- __str__(self): 

Méthode pour afficher le plateau sous forme de chaîne de caractères.
Les O sont affichés en vert et les X en rouges.
Les O représentent les coups jouer.
Les X représentent les cases ne pouvant plus être joués.

- joue(self, x, y): 

Méthode qui permet de jouer un coup en plaçant un "O" à la position (x, y) sur le plateau. Elle met également à jour les cases "X" sur la ligne, la colonne et les diagonales de la case jouée.

- cases_vides(self): 

Méthode qui renvoie la liste des coordonnées des cases vides du plateau.

- afini(self): 

Méthode qui vérifie si le plateau est entièrement rempli, c'est-à-dire s'il n'y a plus de cases vides.
Renvoie True si remplie
Renvoie False si non remplie

- compteur_O(self): 

Méthode qui retourne le nombre de "O" sur le plateau.

- symetrique(self, other): 

Méthode qui vérifie si le plateau actuel est symétrique par rapport à un autre plateau.

- minimax(self, coupsPossibles, alpha, beta): 

Algorithme Minimax avec élagage alpha-bêta pour trouver le meilleur coup à jouer. Il évalue chaque coup possible en appelant evaluate_coup() et retourne le meilleur coup trouvé.

- evaluate_coup(self, coup, alpha, beta): 
Méthode qui évalue un coup donné en calculant un score basé sur heuristique_formation_L. Elle utilise la méthode heuristique_formation_L() pour évaluer la formation de "L" optimales.

- heuristique_formation_L(self, position, coup): 
Méthode qui calcule un score basé sur la formation de "L" optimales autour du coup donné.
Elle permet évalue la formation de "L" optimales autour d'un coup donné. Elle examine les cases voisines du coup et attribue un score en fonction de l'efficacité de la formation en L.

- nouveaux_coups(self, x, y): 
Méthode qui renvoie une liste des nouveaux coups possibles après avoir joué un coup à la position (x, y).

### Définition 

- Heuristique :

Qui procède par approches successives en éliminant progressivement les alternatives et en ne conservant qu'une gamme restreinte de solutions tendant vers celle qui est optimale. -> https://www.cnrtl.fr/definition/heuristique#:~:text=Qui%20procède%20par%20approches%20successives,Méthode%20heuristique%20p.

- Elagage alpha-bêta :
L'élagage alpha-bêta est une technique utilisée dans l'algorithme Minimax pour réduire le nombre de nœuds explorés dans l'arbre de recherche. L'idée principale est de ne pas explorer les branches de l'arbre qui ne sont pas susceptibles de donner de meilleures solutions que celles déjà trouvées. Cela permet d'optimiser l'algorithme du minimax classique et de gagner en temps de calcul.



In [59]:
import numpy as np
from copy import deepcopy
from concurrent.futures import ThreadPoolExecutor

class Plateau(object):
    def __init__(self, n):
        self.contenu = np.empty((n, n), dtype=object)
        self.tailleM = n
        
    def __str__(self):
        tailleM = self.tailleM
        res = ""
        for i in range(tailleM):
            res += "\n+" + "".join(["---+" for _ in range(tailleM)]) + "\n| "
            for j in range(tailleM):
                valeur = self.contenu[i][j]
                if valeur == "O":
                    res += "\033[92m" + str(valeur) + "\033[0m | " 
                elif valeur == "X":
                    res += "\033[91m" + str(valeur) + "\033[0m | " 
                else:
                    res += "  | "
        res += "\n+" + "".join(["---+" for _ in range(tailleM)]) + "\n "
        return res

    def joue(self, x, y):
        tailleM = self.tailleM
        if self.contenu[x][y] is None:
            self.contenu[x][y] = "O"
            for i in range(tailleM):
                if i != x:
                    self.contenu[i][y] = "X"
                if i != y:
                    self.contenu[x][i] = "X"
            for i in range(1, tailleM):
                if x + i < tailleM and y + i < tailleM:
                    if x + i != x or y + i != y:
                        self.contenu[x + i][y + i] = "X"
                if x - i >= 0 and y - i >= 0:
                    if x - i != x or y - i != y:
                        self.contenu[x - i][y - i] = "X"
                if x + i < tailleM and y - i >= 0:
                    if x + i != x or y - i != y:
                        self.contenu[x + i][y - i] = "X"
                if x - i >= 0 and y + i < tailleM:
                    if x - i != x or y + i != y:
                        self.contenu[x - i][y + i] = "X"

    def cases_vides(self):
        cases_vide = []
        for i in range(self.tailleM):
            for j in range(self.tailleM):
                if self.contenu[i][j] is None:
                    cases_vide.append((i, j))
        return cases_vide

    def afini(self):
        i, j = 0, 0
        while j < self.tailleM:
            if self.contenu[i][j] is None:
                return False
            else:
                i += 1
                if i == self.tailleM:
                    i = 0
                    j += 1
        return True
    
    def compteur_O(self):
        compteur=0
        for i in range(self.tailleM):
            for j in range(self.tailleM):
                if self.contenu[i][j]=="O":
                    compteur+=1
        return compteur
    
    def symetrique(self, other):
        tailleM = self.tailleM
        for i in range(tailleM):
            for j in range(tailleM):
                if self.contenu[i][j] != other.contenu[tailleM-i-1][tailleM-j-1]:
                    return False
        return True

    def minimax(self, coupsPossibles, alpha, beta):
        if self.afini() or not coupsPossibles:
            return self.compteur_O()
        v = float('-inf')
        best_coup = None
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = {executor.submit(self.evaluate_coup, coup, alpha, beta): coup for coup in coupsPossibles}
            for future in futures:
                score = future.result()
                if isinstance(score, int) or isinstance(score, float):
                    if score > v:
                        v = score
                        best_coup = futures[future]
                else:
                    if score[1] > v:
                        v = score[1]
                        best_coup = score[0]
                alpha = max(alpha, v)
                if alpha >= beta:
                    break
        return best_coup
        
    def evaluate_coup(self, coup, alpha, beta):
        newPos = deepcopy(self)
        newPos.joue(*coup)
        score = 0
        score += self.heuristique_formation_L(newPos, coup)
        return score

    def heuristique_formation_L(self, position, coup):
        score = 0
        for i, j in [(1, 2), (2, 1), (1, -2), (-2, 1), (-1, 2), (2, -1), (-1, -2), (-2, -1)]:
            if 0 <= coup[0] + i < position.tailleM and 0 <= coup[1] + j < position.tailleM:
                if position.contenu[coup[0] + i][coup[1] + j] == "O":
                    score += 1
        return score

    def nouveaux_coups(self, x, y):
        nouveaux = []
        for i in range(self.tailleM):
            for j in range(self.tailleM):
                if self.contenu[i][j] is None and (i != x or j != y):
                    nouveaux.append((i, j))
        return nouveaux



### Test pour un plateau de 3

Résultat attendu 2 "O"

In [60]:
if __name__ == "__main__":
    taille_plateau = int(input("Entrez la taille du plateau : "))
    jeu = Plateau(taille_plateau)
    coups_possibles = jeu.cases_vides()
    start_i, start_j = 0, 1
    jeu.joue(start_i, start_j)
    while True:
        if jeu.afini():
            print(jeu)
            print("Nombre de O placé sur le plateau : " + str(jeu.compteur_O()))
            break
        else:
            coup_choisi = jeu.minimax(coups_possibles, float('-inf'), float('inf'))
            jeu.joue(*coup_choisi)
            print(jeu)
            coups_possibles = jeu.nouveaux_coups(*coup_choisi)

Entrez la taille du plateau : 3

+---+---+---+
| X | O | X | 
+---+---+---+
| X | X | X | 
+---+---+---+
| O | X | X | 
+---+---+---+
 

+---+---+---+
| X | O | X | 
+---+---+---+
| X | X | X | 
+---+---+---+
| O | X | X | 
+---+---+---+
 
Nombre de O placé sur le plateau : 2


### Test pour un plateau de 4
Résultat attendu 4 "O"

In [61]:
if __name__ == "__main__":
    taille_plateau = int(input("Entrez la taille du plateau : "))
    jeu = Plateau(taille_plateau)
    coups_possibles = jeu.cases_vides()
    start_i, start_j = 0, 1
    jeu.joue(start_i, start_j)
    while True:
        if jeu.afini():
            print(jeu)
            print("Nombre de O placé sur le plateau : " + str(jeu.compteur_O()))
            break
        else:
            coup_choisi = jeu.minimax(coups_possibles, float('-inf'), float('inf'))
            jeu.joue(*coup_choisi)
            print(jeu)
            coups_possibles = jeu.nouveaux_coups(*coup_choisi)

Entrez la taille du plateau : 4

+---+---+---+---+
| X | O | X | X | 
+---+---+---+---+
| X | X | X | O | 
+---+---+---+---+
|   | X | X | X | 
+---+---+---+---+
|   | X |   | X | 
+---+---+---+---+
 

+---+---+---+---+
| X | O | X | X | 
+---+---+---+---+
| X | X | X | O | 
+---+---+---+---+
| O | X | X | X | 
+---+---+---+---+
| X | X |   | X | 
+---+---+---+---+
 

+---+---+---+---+
| X | O | X | X | 
+---+---+---+---+
| X | X | X | O | 
+---+---+---+---+
| O | X | X | X | 
+---+---+---+---+
| X | X | O | X | 
+---+---+---+---+
 

+---+---+---+---+
| X | O | X | X | 
+---+---+---+---+
| X | X | X | O | 
+---+---+---+---+
| O | X | X | X | 
+---+---+---+---+
| X | X | O | X | 
+---+---+---+---+
 
Nombre de O placé sur le plateau : 4


### Test pour un plateau de 5
Résultat attendu 5 "O"

In [50]:
if __name__ == "__main__":
    taille_plateau = int(input("Entrez la taille du plateau : "))
    jeu = Plateau(taille_plateau)
    coups_possibles = jeu.cases_vides()
    start_i, start_j = 0, 1
    jeu.joue(start_i, start_j)
    while True:
        if jeu.afini() or not coups_possibles:
            print(jeu)
            print("Nombre de O placé sur le plateau : " + str(jeu.compteur_O()))
            break
        else:
            coup_choisi = jeu.minimax(coups_possibles, float('-inf'), float('inf'))
            jeu.joue(*coup_choisi)
            print(jeu)
            coups_possibles = jeu.nouveaux_coups(*coup_choisi)

Entrez la taille du plateau : 5

+---+---+---+---+---+
| X | O | X | X | X | 
+---+---+---+---+---+
| X | X | X | O | X | 
+---+---+---+---+---+
|   | X | X | X | X | 
+---+---+---+---+---+
|   | X |   | X | X | 
+---+---+---+---+---+
| X | X |   | X |   | 
+---+---+---+---+---+
 

+---+---+---+---+---+
| X | O | X | X | X | 
+---+---+---+---+---+
| X | X | X | O | X | 
+---+---+---+---+---+
| O | X | X | X | X | 
+---+---+---+---+---+
| X | X |   | X | X | 
+---+---+---+---+---+
| X | X | X | X |   | 
+---+---+---+---+---+
 

+---+---+---+---+---+
| X | O | X | X | X | 
+---+---+---+---+---+
| X | X | X | O | X | 
+---+---+---+---+---+
| O | X | X | X | X | 
+---+---+---+---+---+
| X | X | O | X | X | 
+---+---+---+---+---+
| X | X | X | X |   | 
+---+---+---+---+---+
 

+---+---+---+---+---+
| X | O | X | X | X | 
+---+---+---+---+---+
| X | X | X | O | X | 
+---+---+---+---+---+
| O | X | X | X | X | 
+---+---+---+---+---+
| X | X | O | X | X | 
+---+---+---+---+---+
| X | X | X | X